In [4]:
from pyspark.sql import SparkSession

app_name = "Absence Analysis"

spark = SparkSession.builder.master("local[*]").appName(app_name).getOrCreate()

In [11]:
# Load the data
df = spark.read.csv('../data/raw/Absence_3term201819_nat_reg_la_sch.csv')
df_alias = df.alias('df_alias')

In [20]:
from pyspark.sql.functions import count, when, isnan, col

# split into categorical and numerical columns
cat_cols = list()
num_cols = list()

for i, col_ in enumerate(df.dtypes):
    print(df.select(col_[0]).dtypes)
    if df_alias.dtypes[i][1] == 'string':
        cat_cols.append(col_)
    else:
        num_cols.append(col_)


[('_c0', 'string')]
[('_c1', 'string')]
[('_c2', 'string')]
[('_c3', 'string')]
[('_c4', 'string')]
[('_c5', 'string')]
[('_c6', 'string')]
[('_c7', 'string')]
[('_c8', 'string')]
[('_c9', 'string')]
[('_c10', 'string')]
[('_c11', 'string')]
[('_c12', 'string')]
[('_c13', 'string')]
[('_c14', 'string')]
[('_c15', 'string')]
[('_c16', 'string')]
[('_c17', 'string')]
[('_c18', 'string')]
[('_c19', 'string')]
[('_c20', 'string')]
[('_c21', 'string')]
[('_c22', 'string')]
[('_c23', 'string')]
[('_c24', 'string')]
[('_c25', 'string')]
[('_c26', 'string')]
[('_c27', 'string')]
[('_c28', 'string')]
[('_c29', 'string')]
[('_c30', 'string')]
[('_c31', 'string')]
[('_c32', 'string')]
[('_c33', 'string')]
[('_c34', 'string')]
[('_c35', 'string')]
[('_c36', 'string')]
[('_c37', 'string')]
[('_c38', 'string')]
[('_c39', 'string')]
[('_c40', 'string')]
[('_c41', 'string')]
[('_c42', 'string')]
[('_c43', 'string')]
[('_c44', 'string')]
[('_c45', 'string')]
[('_c46', 'string')]
[('_c47', 'string')]
[(